In [0]:
import os
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
import time

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers, optimizers, utils

In [0]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras.optimizers import RMSprop,Adam
from keras.optimizers import SGD
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [0]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
binary_flder = '/content/drive/Shared drives/DS8013/Data/Binary/'

In [0]:
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications


In [0]:
# input_tensor = Input(shape=(150,150,3))

In [0]:
base_model = InceptionV3(weights='imagenet', include_top=False,input_shape=(150, 150,3))

In [0]:
CLASSES = 10
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)
predictions = Dense(CLASSES, activation='softmax')(x)
model_V3 = Model(inputs=base_model.input, outputs=predictions)

In [0]:
for layer in base_model.layers:
    layer.trainable = False

In [0]:
model_V3.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv2d_471 (Conv2D)             (None, 74, 74, 32)   864         input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_471 (BatchN (None, 74, 74, 32)   96          conv2d_471[0][0]                 
__________________________________________________________________________________________________
activation_471 (Activation)     (None, 74, 74, 32)   0           batch_normalization_471[0][0]    
____________________________________________________________________________________________

In [0]:
# Compile model
# opt = RMSprop(lr=0.0001)
opt = SGD(lr=0.01, momentum=0.9)
model_V3.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2020-04-23 17:54:39--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.14.112, 2607:f8b0:4007:808::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.14.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   127MB/s    in 0.7s    

2020-04-23 17:54:40 (127 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [0]:
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [0]:
pre_trained_model = InceptionV3(
    input_shape=(150, 150, 3), include_top=False, weights=None)
pre_trained_model.load_weights(local_weights_file)

In [0]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [0]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 7, 7, 768)


In [0]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
#x = layers.Dense(1024, activation='relu')(last_output)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
# Add a final sigmoid layer for classification
x = layers. Dense(10, activation='softmax')(x)
#x = layers.Dense(1, activation='sigmoid')(x)

# Configure the model
model = Model(pre_trained_model.input, x)


In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 74, 74, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 74, 74, 32)   96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 74, 74, 32)   0           batch_normalization_94[0][0]     
____________________________________________________________________________________________

In [0]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.0001),
              metrics=['acc'])

In [0]:
opt = SGD(lr=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
path = '/content/drive/Shared drives/DS8013/Data/EuroSAT/2750/'

In [0]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

In [0]:
# Note that the validation data should not be augmented!
val_datagen = ImageDataGenerator(rescale=1./255,
                                 validation_split=0.2)

In [0]:
train_generator = train_datagen.flow_from_directory(
        path, # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        subset='training')

Found 21600 images belonging to 10 classes.


In [0]:
validation_generator = val_datagen.flow_from_directory(
        path,
        target_size=(150, 150),
        batch_size=20,
        subset='validation')

Found 5400 images belonging to 10 classes.


In [0]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=2,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

In [0]:
history = model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=2,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

Epoch 1/2
100/100 - 1403s - loss: 0.7830 - acc: 0.7495 - val_loss: 1.4363 - val_acc: 0.6180
Epoch 2/2
100/100 - 1296s - loss: 0.6367 - acc: 0.8035 - val_loss: 0.4956 - val_acc: 0.8350


In [0]:
hist = model_V3.fit_generator(
      train_generator,
      steps_per_epoch=320,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=64,
      verbose=2)

Epoch 1/5
 - 2336s - loss: 1.4811 - accuracy: 0.5827 - val_loss: 2.9036 - val_accuracy: 0.5117
Epoch 2/5
 - 2260s - loss: 1.3701 - accuracy: 0.6536 - val_loss: 1.1290 - val_accuracy: 0.5008
Epoch 3/5
 - 2228s - loss: 1.3083 - accuracy: 0.6819 - val_loss: 2.4723 - val_accuracy: 0.4242
Epoch 4/5


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 114 could not be retrieved. It could be because a worker has died.
  UserWarning)


 - 1037s - loss: 1.4781 - accuracy: 0.6681 - val_loss: 2.3766 - val_accuracy: 0.4203
Epoch 5/5
 - 86s - loss: 1.4365 - accuracy: 0.6820 - val_loss: 3.7580 - val_accuracy: 0.4969


In [0]:
hist = model_V3.fit(
      train_generator,
      steps_per_epoch=320,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=64,
      verbose=2)

Epoch 1/5
 - 46s - loss: 1.4807 - accuracy: 0.6820 - val_loss: 2.4208 - val_accuracy: 0.4344
Epoch 2/5
 - 45s - loss: 1.5031 - accuracy: 0.6877 - val_loss: 2.7179 - val_accuracy: 0.4953
Epoch 3/5
 - 46s - loss: 1.4479 - accuracy: 0.6953 - val_loss: 2.1872 - val_accuracy: 0.5117
Epoch 4/5
 - 45s - loss: 1.5143 - accuracy: 0.6900 - val_loss: 1.5569 - val_accuracy: 0.5305
Epoch 5/5
 - 46s - loss: 1.5018 - accuracy: 0.6914 - val_loss: 2.3746 - val_accuracy: 0.4539
